In [1]:
!pip install rdkit catboost
!python -m pip install git+https://github.com/EBjerrum/molvecgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00
  Cloning https://github.com/EBjerrum/molvecgen to /tmp/pip-req-build-h_5rdshr
  Running command git clone --filter=blob:none --quiet https://github.com/EBjerrum/molvecgen /tmp/pip-req-build-h_5rdshr
  Resolved https://github.com/EBjerrum/molvecgen to commit f81d5aade18bea60882f5845877f6283366bbe91
  Preparing metadata (setup.py) ... done
  Created wheel for molvecgen: filename=molvecgen-0.1-py3-none-any.whl size=11357 sha256=9d78db4af4c8e0a30f73ccb334b31c421a6e9a7dc0c3212d52d884a1f7f411c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-5_vuy3k4/wheels/1c/de/14/fd9892b48271e63e56b00c7015c754ae6189e258fa313a7c63
Successfully built molvecgen


In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole
from sklearn.preprocessing import FunctionTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import cross_val_score
from molvecgen.vectorizers import SmilesVectorizer

In [3]:
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [246]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Atomic hack 2023/Копия 1400.xlsx')
df = df.drop(columns = 'Pictures')
df['SI'] = df['CC50-MDCK, mmg/ml']/df['IC50, mmg/ml']
df['S_leng'] = df['SMILES'].str.len()

In [247]:
df.drop_duplicates(inplace=True)

In [248]:
df.shape

(1459, 10)

In [ ]:
def RDKfingerPrint(mol_smi, **kwargs):
    mol = Chem.MolFromSmiles(mol_smi)
    desc_vec = np.zeros((1,), dtype=int)
    DataStructs.ConvertToNumpyArray(AllChem.RDKFingerprint(mol, **kwargs), desc_vec)
    return desc_vec

# Применить функцию RDKfingerPrint ко всем строкам в столбце 'SMILES' и создать новые столбцы
df[['RDKFP_' + str(i) for i in range(2048)]] = df['SMILES'].apply(lambda x: pd.Series(RDKfingerPrint(x, maxPath=5)))

In [250]:
# создаём фингерпринты, находим дубликаты. Итерируемся по дубликатам, рассчитываем расстояние,
# изменяем таблицу с дубликатами, дропаем все дубликаты, вставляем измен1нные дубликаты в общую таблицу
df['finger'] = df['SMILES'].apply(lambda x: AllChem.RDKFingerprint(Chem.MolFromSmiles(x)))
duplicates = df[df.duplicated(subset='Title', keep='first')]

for i in range(len(duplicates)):
  distances = []
  fps_1 = duplicates.iloc[i]['finger']
  for j in range(len(df)):
    fps_2 = df.iloc[j]['finger']
    if fps_1 != fps_2:
      dist = DataStructs.FingerprintSimilarity(fps_1, fps_2)
      distances.append(dist)
  duplicates.loc[duplicates.index[i], 'IC50, mmg/ml'] = df.iloc[np.argmax(distances)]['IC50, mmg/ml']
  duplicates.loc[duplicates.index[i], 'CC50-MDCK, mmg/ml'] = df.iloc[np.argmax(distances)]['CC50-MDCK, mmg/ml']
  duplicates.loc[duplicates.index[i], 'SI'] = df.iloc[np.argmax(distances)]['SI']
df.drop_duplicates(subset='Title', keep=False, inplace=True)
df = pd.concat([df, duplicates]).drop('finger', axis=1)

[09:07:55] Conflicting single bond directions around double bond at index 55.
[09:07:55]   BondStereo set to STEREONONE and single bond directions set to NONE.
<ipython-input-250-c9f85d71ef07>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['finger'] = df['SMILES'].apply(lambda x: AllChem.RDKFingerprint(Chem.MolFromSmiles(x)))


In [251]:
# удаление выбросов
df = df[df['SI'] < 100]
df = df[df['Polar SA'] < 300]
df = df[df['Molecular weight'] < 1000]
df = df[df['S_leng'] < 200]

In [253]:
def mol_dsc_calc(mols):
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in mols)

# список конституционных и физико-химических дескрипторов из библиотеки RDKit
descriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
               "NHOHCount": Descriptors.NHOHCount,
               "NOCount": Descriptors.NOCount,
               "NumHAcceptors": Descriptors.NumHAcceptors,
               "NumHDonors": Descriptors.NumHDonors,
               "NumHeteroatoms": Descriptors.NumHeteroatoms,
               "NumRotatableBonds": Descriptors.NumRotatableBonds,
               "NumValenceElectrons": Descriptors.NumValenceElectrons,
               "NumAromaticRings": Descriptors.NumAromaticRings,
               "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
               "RingCount": Descriptors.RingCount,
               "MW": Descriptors.MolWt,
               "LogP": Descriptors.MolLogP,
               "MR": Descriptors.MolMR,
               "TPSA": Descriptors.TPSA}

# sklearn трансформер для использования в конвейерном моделировании
descriptors_transformer = FunctionTransformer(mol_dsc_calc)
X_descr = descriptors_transformer.transform(df['SMILES'])
X_descr.head()

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,18,0,2,2,0,2,5,104,0,0,2,250.430,3.6154,79.3190,15.60
1,16,0,2,2,0,2,3,92,0,0,2,222.376,2.8352,70.0850,15.60
2,17,0,3,3,0,3,3,98,0,0,2,239.359,2.3060,67.6630,29.54
3,19,0,2,1,0,2,5,110,0,0,2,265.465,3.7600,83.6644,12.36
4,20,0,2,1,0,2,6,116,0,0,2,279.492,4.1501,88.2814,12.36


In [254]:
df.head()

,Title,"IC50, mmg/ml","CC50-MDCK, mmg/ml",SI,Molecular weight,Hydrogen bond acceptors,Hydrogen bond donors,Polar SA,SMILES,S_leng,...,RDKFP_2038,RDKFP_2039,RDKFP_2040,RDKFP_2041,RDKFP_2042,RDKFP_2043,RDKFP_2044,RDKFP_2045,RDKFP_2046,RDKFP_2047
2,1008-Ya-187,9.9,144.0,14.545455,250.431,1,0,15.60,CCN(CC)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,41,...,0,0,0,0,0,0,0,0,0,1
3,1009-As-106,8.3,500.0,60.240964,222.377,1,0,15.60,CN(C)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,39,...,0,0,0,0,0,0,0,0,0,1
4,1010-Ya-208,39.4,143.0,3.629442,239.361,2,0,29.54,CN(C)CC(=O)O[C@H]1C[C@H](CC2)C(C)(C)[C@@]12C,44,...,0,0,0,0,0,0,0,0,0,1
5,1011-As-83,25.8,500.0,19.379845,265.466,1,0,12.36,CC[N+](C)(CC)CC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,47,...,0,0,0,0,0,0,0,0,0,1
6,1012-Ya-201,39.4,498.0,12.639594,279.493,1,0,12.36,CC[N+](CC)(CC)CC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,48,...,0,0,0,0,0,0,0,0,0,1


In [255]:
df.reset_index(inplace=True)

In [256]:
df = pd.concat([df, X_descr], axis=1)  # add descriptor

In [257]:
# shuffle
df = df.sample(frac=1)

In [258]:
df_w = df[['IC50, mmg/ml', 'CC50-MDCK, mmg/ml', 'SMILES']]
df_v = df[['IC50, mmg/ml', 'CC50-MDCK, mmg/ml', 'SI', 'Molecular weight',
       'Hydrogen bond acceptors', 'Hydrogen bond donors', 'Polar SA', 'S_leng']]

In [259]:
PandasTools.AddMoleculeColumnToFrame(df_w,'SMILES','Molecule')
df_w[["SMILES","Molecule"]]

/usr/local/lib/python3.10/dist-packages/rdkit/Chem/PandasTools.py:371: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[molCol] = frame[smilesCol].map(Chem.MolFromSmiles)


,SMILES,Molecule
1034,OC[C@@H]1[C@@H](O)[C@H](O)[C@@H](O)[C@@H](O1)O...,<rdkit.Chem.rdchem.Mol object at 0x780789098ac0>
644,CC(=O)C(C1=O)=C(O)C=C2Oc(c3[C@@]12C)c(c(O)c(C)...,<rdkit.Chem.rdchem.Mol object at 0x7807869306d0>
554,Cc1ccc(cc1)/N=C/C=C/c2cc(on2)-c3ccccc3,<rdkit.Chem.rdchem.Mol object at 0x780786930890>
1237,CC(C)(C)/C(N)=N/OC(=O)[C@]12C(=O)C[C@H](C1(C)C...,<rdkit.Chem.rdchem.Mol object at 0x780786930a50>
172,C1C=C(C)[C@@H](O)[C@@H]([C@@H]12)O[C@@H](C[C@@...,<rdkit.Chem.rdchem.Mol object at 0x7807869309e0>
...,...,...
197,OCc1cn(nn1)CC/N=C(\[C@@]23C)C[C@H](C2(C)C)CC3,<rdkit.Chem.rdchem.Mol object at 0x78078a5a4740>
154,C[C@]12C(C)(C)[C@@H](CC2)C[C@@H]1OC(=O)CCN3CCN...,<rdkit.Chem.rdchem.Mol object at 0x78078a5a47b0>
445,c1ccccc1-c(c(c2=O)C(=O)OCC)oc(c23)c(F)c(c(F)c3...,<rdkit.Chem.rdchem.Mol object at 0x78078a5a4820>
490,c1coc(c12)C[C@H]3[C@H]([C@@H]2C(=O)O)C(=O)N4[C...,<rdkit.Chem.rdchem.Mol object at 0x78078a5a4890>


In [260]:
smivec = SmilesVectorizer(pad=1, leftpad=True, canonical=False, augment=True)
smivec.fit(df_w.Molecule.values)

In [261]:
class SMILESMolDataset(Dataset):
    def __init__(self, molecules, y, vectorizer):
        self.molecules = molecules
        self.y = y
        self.vectorizer = vectorizer
    def __len__(self):
        return len(self.molecules)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        mols = self.molecules[idx]
        sample = self.vectorizer.transform([mols])[0]
        label = self.y[idx]
        return sample, label

In [262]:
y = df_w['CC50-MDCK, mmg/ml'].values.reshape((-1,1))
X = df_w.Molecule.values
train_dataset = SMILESMolDataset(X, y, smivec)

new_dataset = []

for item in train_dataset:
    flattened_item = item[0][0].flatten()
    new_dataset.append([*flattened_item])
smiles = pd.DataFrame(new_dataset)

result_df = df.join(smiles).drop(columns = ['Title', 'SMILES'])
result_df = result_df.drop(columns = ['IC50, mmg/ml', 'CC50-MDCK, mmg/ml', 'SI', 'index'])

In [263]:
y1 = df_w['IC50, mmg/ml'].values.reshape((-1,1))
y2 = df_w['CC50-MDCK, mmg/ml'].values.reshape((-1,1))

In [264]:
result_df.shape

(1260, 2108)

In [266]:
result_df.head()

,Molecular weight,Hydrogen bond acceptors,Hydrogen bond donors,Polar SA,S_leng,RDKFP_0,RDKFP_1,RDKFP_2,RDKFP_3,RDKFP_4,...,30,31,32,33,34,35,36,37,38,39
1034,418.404,9,5,145.91,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
644,400.413,7,4,171.21,61,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
554,288.352,3,0,38.39,38,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1237,280.370,4,0,81.75,50,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
172,364.443,6,2,77.38,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
cat_model1 = CatBoostRegressor(verbose = 500)
cat_model2 = CatBoostRegressor(verbose = 500)

In [270]:
result_df.shape

(1260, 2108)

In [272]:
num = int(len(result_df) * 0.75)
X_train = result_df.iloc[:num]
X_test = result_df.iloc[num:len(result_df)]
y1_train = y1[:num]
y2_train = y2[:num]
y1_test = y1[num:]
y2_test = y2[num:]

In [280]:
# IC50
cat_model1.load_model('weights.bin')
y_pred1 = cat_model1.predict(X_test.drop(['Molecular weight', 'Hydrogen bond acceptors', 'Hydrogen bond donors', 'Polar SA'], axis=1))

In [276]:
# CC50
cat_model2.fit(X_train, y2_train)
y_pred2 = cat_model2.predict(X_test)

Learning rate set to 0.040579
0:	learn: 146.4780548	total: 118ms	remaining: 1m 57s
500:	learn: 57.5901986	total: 1m 6s	remaining: 1m 6s
999:	learn: 40.5833634	total: 2m 7s	remaining: 0us


In [277]:
mean_squared_error(y2_test, y_pred2, squared=False)

105.92430537077415

In [282]:
mean_squared_error(y1_test, y_pred1, squared=False)

92.08177185528345

In [281]:
rmse = mean_squared_error(y2_test/y1_test, y_pred2/y_pred1, squared=False)
rmse

62.19948147007722